In [2]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

text_splitter = CharacterTextSplitter()

def split_the_pdf_texts(file_path: str) -> list[str]:
    with open(file_path) as f:
        how_to_win_friends = f.read()
        texts = text_splitter.split_text(how_to_win_friends)
    return texts

c:\Users\DNDT\Documents\venv\sarjana-env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [23]:
from pathlib import Path

from llama_index.core import Document
from langchain_community.document_loaders import PyPDFLoader
def load_pdf(path: Path) -> list[Document]:
        pdf_loader = PyPDFLoader(path)
        documents = pdf_loader.load_and_split()
        return documents

In [24]:

docs = load_pdf("data/llama2.pdf")
print(type(docs))
print(type(docs[0]))


<class 'list'>
<class 'langchain_core.documents.base.Document'>


## Map reduce method

In [25]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

def progressive_summarize(docs: list[Document]):
    # with a custom prompt
    prompt_template = """Write a concise summary of the following:

    {text}

    CONSCISE SUMMARY IN BULLET POINTS:"""

    PROMPT = PromptTemplate(template=prompt_template,input_variables=["text"])
    ## with intermediate steps
    chain = load_summarize_chain(OpenAI(temperature=0),chain_type="map_reduce",return_intermediate_steps=True,map_prompt=PROMPT,combine_prompt=PROMPT)

    output_summary = chain({"input_documents": docs}, return_only_outputs=True)
    wrapped_text = textwrap.fill(output_summary['output_text'],width=100,break_long_words=False,replace_whitespace=False)
    print(wrapped_text)
    return output_summary


In [26]:
summarized = progressive_summarize(docs=docs)



- Llama 2-Chat is a collection of pretrained and fine-tuned large language models optimized for
dialogue use cases.
- It outperforms open-source chat models on most benchmarks and has been
evaluated for helpfulness and safety.
- The authors provide a detailed description of their approach
to fine-tuning and safety improvements.
- The goal is to enable the community to build on their work
and contribute to the responsible development of LLMs.
- The paper discusses the pretraining, fine-
tuning, and safety measures used in the development of Llama 2 and Llama 2-Chat.
- The models were
evaluated on various benchmarks and showed strong performance.
- Safety measures were implemented in
both the pretraining and fine-tuning stages.
- The paper also shares observations and insights from
the development process.
- A responsible release strategy and code examples are provided for safe
deployment.
- The study compares two reinforcement learning algorithms and explores the use of Ghost
Attentio

In [27]:
summarized

{'intermediate_steps': ['\n- Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs).\n- The Llama 2-Chat models are optimized for dialogue use cases.\n- Llama 2-Chat outperforms open-source chat models on most benchmarks.\n- The models have been evaluated for helpfulness and safety and may be a suitable substitute for closed-source models.\n- The authors provide a detailed description of their approach to fine-tuning and safety improvements.\n- The goal is to enable the community to build on their work and contribute to the responsible development of LLMs.',
  ' \n\n• The paper discusses the pretraining, fine-tuning, and safety measures used in the development of the Llama 2 chatbot.\n• Pretraining involved using large amounts of data and training details to create a pretrained model.\n• The pretrained model was evaluated using the Llama 2 Pretrained Model Evaluation method.\n• Fine-tuning was done through supervised fine-tuning and reinforcement learning wit

In [28]:
summarized.keys()

dict_keys(['intermediate_steps', 'output_text'])